## Homework
Your homework is to create a new notebook modeled after this one, where:

- You've put all of your cleaning and prepration steps into a functions you load and use in a module.
- You create and train a simple appropriate model for your task.
- You assess the performance using the appropriate metric for your task.
- Try a few different hyper-parameters to see if you can get better performance.
- Compare the performance with your previous best non Deep Neural Network model. This probably will mean you copy and paste your training from your other notebook into this notebook.

### Load Data

In [1]:
# Load the data

import numpy as np
import pandas as pd

df = pd.read_csv('../../../playground-series-s4e3/train.csv', index_col='id')
df.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
id,,,,,,,,,,,,,,,,,,,,,
0,584,590,909972,909977,16,8,5,2274,113,140,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,808,816,728350,728372,433,20,54,44478,70,111,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,39,192,2212076,2212144,11388,705,420,1311391,29,141,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,781,789,3353146,3353173,210,16,29,3202,114,134,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,1540,1560,618457,618502,521,72,67,48231,82,111,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1


In [2]:
# define the lists of column names

column_names = df.columns.tolist()
#print("Column names:\n\n", column_names, end='\n\n')
features_all = column_names[:27]
#print('All Features:\n\n', features_all, end='\n\n')
features_cat = [features_all[11],features_all[12],features_all[20]]
#print('Categorical features:\n\n', features_cat, end='\n\n')
features_num = [i for i in features_all if i not in features_cat]
#print('Numerical features:\n\n', features_num, end='\n\n')
defect_types = column_names[27:34]
#print('Defect types:\n\n', defect_types)

### Create label encoded defect column

In [3]:
# Create single categorical Defect column to encode target more conveniently 
df['Defect'] = df[defect_types].idxmax(axis=1)
df.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults,Defect
id,,,,,,,,,,,,,,,,,,,,,
0,584,590,909972,909977,16,8,5,2274,113,140,...,-0.0104,0.1417,0,0,0,1,0,0,0,Stains
1,808,816,728350,728372,433,20,54,44478,70,111,...,-0.2997,0.9491,0,0,0,0,0,0,1,Other_Faults
2,39,192,2212076,2212144,11388,705,420,1311391,29,141,...,-0.0944,1.0000,0,0,1,0,0,0,0,K_Scatch
3,781,789,3353146,3353173,210,16,29,3202,114,134,...,-0.0402,0.4025,0,0,1,0,0,0,0,K_Scatch
4,1540,1560,618457,618502,521,72,67,48231,82,111,...,-0.2455,0.9998,0,0,0,0,0,0,1,Other_Faults


### Drop rows labeled with no defect or multiple defects

In [4]:
# Clean up dulpicate and zero output rows

print(df.shape)

# define rows w/ no defect and rows w/ 2 defects
zero_rows = df[df[defect_types].sum(axis=1)==0].index
multi_rows = df[df[defect_types].sum(axis=1)>1].index

# drop zero rows from df
df.drop(zero_rows, axis=0, inplace=True)
#drop multi defect rows from df
df.drop(multi_rows, axis=0, inplace=True)

# Reset the index
df.reset_index(inplace=True, drop=True)

print(df.shape)

(19219, 35)
(18380, 35)


### Apply scaling and transformation

In [6]:
from sklearn.preprocessing import MinMaxScaler
from scipy.special import boxcox1p
from scipy.stats import skew

skewed_features = df[features_num].apply(lambda x: skew(x)).sort_values(ascending=False)

skewed_features_df = pd.DataFrame(skewed_features, columns={'Skew'})

lam = 0.15
cols = skewed_features_df.index

# Perform boxcox1p transformation
scaled_features = pd.DataFrame({c: boxcox1p(df[c] + 1e-6, lam) for c in cols})

# Define second sacaler
mmscaler = MinMaxScaler()

# Perform fit (to find mean, std, min, max) and transform data (to fit in [0,1]) in one move 
scaled_features = pd.DataFrame(mmscaler.fit_transform(scaled_features), columns=scaled_features.columns)

# Recobine scaled feature data frame with the rest
scaled_df = pd.concat([scaled_features, df[features_cat], df[defect_types], df['Defect']], axis=1)

scaled_df.head(10)

,Pixels_Areas,Sum_of_Luminosity,X_Perimeter,Outside_X_Index,Y_Perimeter,Y_Minimum,Y_Maximum,Steel_Plate_Thickness,Maximum_of_Luminosity,Log_X_Index,...,TypeOfSteel_A400,Outside_Global_Index,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults,Defect
0,0.040967,0.097769,0.080080,0.008301,0.080906,0.515431,0.515370,0.095480,0.509560,0.320167,...,1,0.0,0,0,0,1,0,0,0,Stains
1,0.246677,0.292926,0.151512,0.005475,0.403896,0.482949,0.482888,0.308503,0.329169,0.261679,...,0,1.0,0,0,0,0,0,0,1,Other_Faults
2,0.584707,0.651606,0.567057,0.192235,0.811725,0.656387,0.656348,0.000000,0.515195,0.782047,...,1,0.0,0,0,1,0,0,0,0,K_Scatch
3,0.191879,0.116054,0.132839,0.005475,0.304619,0.729136,0.729103,0.000000,0.475014,0.261679,...,1,1.0,0,0,1,0,0,0,0,K_Scatch
4,0.261684,0.299557,0.274505,0.033207,0.440835,0.459768,0.459710,1.000000,0.329169,0.530193,...,1,1.0,0,0,0,0,0,0,1,Other_Faults
5,0.242136,0.298323,0.159736,0.007171,0.288264,0.513671,0.513622,0.246315,0.376061,0.343159,...,0,1.0,0,0,0,0,0,0,1,Other_Faults
6,0.189808,0.240148,0.155704,0.017884,0.257206,0.485048,0.880496,0.000000,0.433020,0.437431,...,1,1.0,0,0,0,0,0,1,0,Bumps
7,0.269275,0.307601,0.209750,0.023876,0.413032,0.361283,0.361213,1.000000,0.322268,0.517180,...,1,1.0,1,0,0,0,0,0,0,Pastry
8,0.104274,0.162004,0.060580,0.004154,0.210765,0.574694,0.574640,0.176178,0.469130,0.222755,...,0,1.0,0,0,0,0,0,1,0,Bumps
9,0.215314,0.267650,0.207196,0.020695,0.362491,0.459768,0.459710,0.700772,0.433020,0.459288,...,1,1.0,0,0,0,0,0,1,0,Bumps


### Feature engineering

In [7]:
# If we do some feature engineering, we can reduce the number of columns in our data
# X and Y Max and Min can ech be combined into X_Range and Y_Range making 4 columns into 2
scaled_df['X_Range'] = scaled_df['X_Maximum'] - scaled_df['X_Minimum']
scaled_df['Y_Range'] = scaled_df['Y_Maximum'] - scaled_df['Y_Minimum']

# Because the perimeter lengths along the X and Y axis are correlated we combine them to make the area of a defect
scaled_df['Defect_Area'] = scaled_df['X_Perimeter'] * scaled_df['Y_Perimeter']

# We can determine the range of luminosity within any one defect using the max and min of luminosity
scaled_df['Luminosity_Range'] = scaled_df['Maximum_of_Luminosity'] - scaled_df['Minimum_of_Luminosity']

# LogOfAreas, Log_X_Index, and Log_Y_Index are all positively correlated, so we can combine them into a single feature
scaled_df['LogA'] = scaled_df['LogOfAreas'] / (scaled_df['Log_X_Index'] * scaled_df['Log_Y_Index'] + 0.000001) # add small amt to avoid dividing by 0

# Drop unnecessary columns
redundant_cols = ['X_Maximum','X_Minimum','Y_Maximum','Y_Minimum','X_Perimeter','Y_Perimeter',
                  'Maximum_of_Luminosity','Minimum_of_Luminosity','LogOfAreas','Log_X_Index','Log_Y_Index']
engineered_df = scaled_df.drop(redundant_cols, axis=1)
display(engineered_df.head())

,Pixels_Areas,Sum_of_Luminosity,Outside_X_Index,Steel_Plate_Thickness,Length_of_Conveyer,Luminosity_Index,Edges_Index,SigmoidOfAreas,Empty_Index,Square_Index,...,Stains,Dirtiness,Bumps,Other_Faults,Defect,X_Range,Y_Range,Defect_Area,Luminosity_Range,LogA
0,0.040967,0.097769,0.008301,0.095480,0.261464,0.788769,0.792951,0.033149,0.500441,0.567409,...,1,0,0,0,Stains,-0.032485,-0.000061,0.006479,-0.346501,1.819014
1,0.246677,0.292926,0.005475,0.308503,0.834211,0.644273,0.825186,0.953735,0.373828,0.312010,...,0,0,0,1,Other_Faults,-0.022867,-0.000062,0.061195,-0.411480,3.916783
2,0.584707,0.651606,0.192235,0.000000,0.340756,0.750985,0.074780,1.000000,0.634956,0.991916,...,0,0,0,0,K_Scatch,0.161278,-0.000039,0.460294,-0.035394,1.671612
3,0.191879,0.116054,0.005475,0.000000,0.316431,0.775688,0.776479,0.378554,0.426262,0.395577,...,0,0,0,0,K_Scatch,-0.023828,-0.000033,0.040465,-0.383254,3.879415
4,0.261684,0.299557,0.033207,1.000000,0.842154,0.674793,0.158418,0.999820,0.641517,0.101169,...,0,0,0,1,Other_Faults,-0.001358,-0.000059,0.121011,-0.448639,1.926101
